in removing jump step we received the following error:

`/home/hnoorazar/remote_sensing_codes/remote_sensing_core.py:72: RuntimeWarning: invalid value encountered in double_scalars
  m = (y2 - y1) / (x2 - x1) # slope`
  
for the following setting:

`python3 ./01_2Yrs_remove_jumps.py NDVI 2015 Walla_Walla 10_cloud`



In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import scipy
import scipy.signal
import os, os.path

from datetime import date
import datetime
import time

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb

import sys

/Users/hn/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc
import remote_sensing_core as rcp

start_time = time.time()

In [3]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/01_NDVI_TS/10_Cloud/00_outliers_removed/"

In [4]:
####################################################################################
###
###      Parameters                   
###
####################################################################################

SF_year = 2015
indeks = "NDVI"

In [ ]:
f_name = "00_noOutlier_Walla_Walla_SF_2015_NDVI.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

In [ ]:
a_df.head(2)

In [ ]:
print (a_df.shape)
print (len(a_df['ID'].unique()))

In [ ]:
an_EE_TS = rc.initial_clean(df = a_df, column_to_be_cleaned = indeks)

In [ ]:
a_df.shape

In [ ]:
an_EE_TS = a_df.copy()
polygon_list = an_EE_TS['ID'].unique()
print(len(polygon_list))

In [ ]:
output_df = pd.DataFrame(data = None,
                         index = np.arange(an_EE_TS.shape[0]), 
                         columns = an_EE_TS.columns)

counter = 0
row_pointer = 0

In [ ]:
a_poly = "185968_WSDA_SF_2015"
curr_field = an_EE_TS[an_EE_TS['ID']==a_poly].copy()
################################################################
# Sort by DoY (sanitary check)
curr_field.sort_values(by=['image_year', 'doy'], inplace=True)
curr_field.reset_index(drop=True, inplace=True)


In [ ]:
dataTMS_jumpie = curr_field.copy()
give_col = "NDVI"
maxjump_perDay = 0.015

In [ ]:
dataTMS = dataTMS_jumpie.copy()
dataTMS = rc.initial_clean(df = dataTMS, column_to_be_cleaned = give_col)

dataTMS.sort_values(by=['image_year', 'doy'], inplace=True)
dataTMS.reset_index(drop=True, inplace=True)
dataTMS['system_start_time'] = dataTMS['system_start_time'] / 1000

thyme_vec = dataTMS['system_start_time'].values.copy()
Veg_indks = dataTMS[give_col].values.copy()

time_diff = thyme_vec[1:] - thyme_vec[0:len(thyme_vec)-1]
time_diff_in_days = time_diff / 86400
time_diff_in_days = time_diff_in_days.astype(int)

Veg_indks_diff = Veg_indks[1:] - Veg_indks[0:len(thyme_vec)-1]
jump_indexes = np.where(Veg_indks_diff > maxjump_perDay)
jump_indexes = jump_indexes[0]


In [ ]:
no_Outlier_TS = rc.correct_big_jumps_1DaySeries(dataTMS_jumpie = curr_field, 
                                                give_col = indeks, 
                                                maxjump_perDay = 0.015)

output_df[row_pointer: row_pointer + curr_field.shape[0]] = no_Outlier_TS.values
counter += 1
row_pointer += curr_field.shape[0]

In [ ]:
no_Outlier_TS.NDVI

In [ ]:
if len(jump_indexes) > 0:    
    for jp_idx in jump_indexes:
        if  Veg_indks_diff[jp_idx] >= (time_diff_in_days[jp_idx] * maxjump_perDay):
            #
            # form a line using the adjacent points of the big jump:
            #
            x1, y1 = thyme_vec[jp_idx-1], Veg_indks[jp_idx-1]
            x2, y2 = thyme_vec[jp_idx+1], Veg_indks[jp_idx+1]
            print (x1)
            print (x2)
            m = np.float(y2 - y1) / np.float(x2 - x1) # slope
            b = y2 - (m*x2)           # intercept

            # replace the big jump with linear interpolation
            Veg_indks[jp_idx] = m * thyme_vec[jp_idx] + b

dataTMS[give_col] = Veg_indks

In [ ]:
output_df = rc.add_human_start_time_by_YearDoY(output_df)
####################################################################################
###
###                   Write the outputs
###
####################################################################################

out_name = output_dir + "01_outlier_n_jump_removed_" + county + "_SF_" + str(SF_year) + "_" + indeks + ".csv"
os.makedirs(output_dir, exist_ok=True)
output_df.to_csv(out_name, index = False)

end_time = time.time()
print(end_time - start_time)

#  Error 

`/data/hydro/users/Hossein/remote_sensing/02_Eastern_WA_EE_TS/2Years/10_cloud//00_outliers_removed/00_noOutlier_Benton_SF_2016_NDVI.csv'`

`python3 ./01_2Yrs_remove_jumps.py NDVI 2016 Benton 10_cloud`

In [ ]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/01_NDVI_TS/10_Cloud/"

f_name = "Eastern_WA_2016_10cloud_selectors.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

In [ ]:
indeks = "NDVI" 
SF_year = "2016" 
county = "Benton"

an_EE_TS = a_df.copy()

In [ ]:
an_EE_TS = an_EE_TS[an_EE_TS['county'] == county] # Filter county
an_EE_TS['SF_year'] = SF_year


In [ ]:
an_EE_TS.shape

In [ ]:
an_EE_TS = rc.initial_clean(df = an_EE_TS, column_to_be_cleaned = indeks)

In [ ]:
an_EE_TS.shape

In [ ]:
an_EE_TS.head(2)

###
### List of unique polygons
###
polygon_list = an_EE_TS['ID'].unique()

print ("Number of unique fields is: ")
print(len(polygon_list))
print ("__________________________________________")


########################################################################################

output_df = pd.DataFrame(data = None,
                         index = np.arange(an_EE_TS.shape[0]), 
                         columns = an_EE_TS.columns)

counter = 0
row_pointer = 0

In [ ]:
for a_poly in polygon_list:
    if (counter % 300 == 0):
        print (counter)
    curr_field = an_EE_TS[an_EE_TS['ID']==a_poly].copy()
    ################################################################
    # Sort by DoY (sanitary check)
    curr_field.sort_values(by=['image_year', 'doy'], inplace=True)
    curr_field.reset_index(drop=True, inplace=True)
    
    # print ("print(curr_field.shape")
    # print(curr_field.shape)
    # print ("__________________________________________")
    ################################################################
    no_Outlier_TS = rc.interpolate_outliers_EVI_NDVI(outlier_input = curr_field, given_col = indeks)

    output_df[row_pointer: row_pointer + curr_field.shape[0]] = no_Outlier_TS.values
    counter += 1
    row_pointer += curr_field.shape[0]


# Error in outlier removal

`File "/home/hnoorazar/remote_sensing_codes/remote_sensing_core.py", line 334, in interpolate_outliers_EVI_NDVI
    if non_outiers[0] > 0 :
IndexError: index 0 is out of bounds for axis 0 with size 0`

`python3 ./00_2Yrs_remove_outliers.py EVI 2015 "Adams" 70_cloud`

In [5]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/" + \
           "01_NDVI_TS/70_Cloud/00_Eastern_WA_withYear/2Years/"

In [6]:
indeks = "EVI" 
SF_year = "2015" 
county = "Adams"
cloud_type = "70_cloud"

f_name = "Eastern_WA_" + str(SF_year) + "_" + cloud_type.split("_")[0] + "cloud_selectors.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

an_EE_TS = a_df.copy()

In [7]:
an_EE_TS.shape

(3584746, 25)

In [8]:
an_EE_TS.county.unique()

array(['Franklin', 'Benton', 'Yakima', 'Spokane', 'Klickitat', 'Stevens',
       'Adams', 'Pend Oreille', 'Okanogan', 'Walla Walla', 'Ferry',
       'Grant', 'Columbia', 'Douglas', 'Chelan', 'Kittitas', 'Lincoln',
       'Whitman', 'Garfield', 'Asotin'], dtype=object)

In [9]:
an_EE_TS = an_EE_TS[an_EE_TS['county'] == county] # Filter county
an_EE_TS['SF_year'] = SF_year

In [10]:
an_EE_TS.shape

(192698, 26)

In [11]:
an_EE_TS = rc.initial_clean(df = an_EE_TS, column_to_be_cleaned = indeks)
an_EE_TS.shape

(12840, 26)

In [12]:
polygon_list = an_EE_TS['ID'].unique()

print ("Number of unique fields is: ")
print(len(polygon_list))
print ("__________________________________________")


Number of unique fields is: 
6395
__________________________________________


In [13]:
output_df = pd.DataFrame(data = None,
                         index = np.arange(an_EE_TS.shape[0]), 
                         columns = an_EE_TS.columns)

counter = 0
row_pointer = 0

a_poly = '88304_WSDA_SF_2015'

# if (counter % 300 == 0):
#     print (counter)
#     print (a_poly)
curr_field = an_EE_TS[an_EE_TS['ID']==a_poly].copy()
################################################################
# Sort by DoY (sanitary check)
curr_field.sort_values(by=['image_year', 'doy'], inplace=True)
curr_field.reset_index(drop=True, inplace=True)

In [14]:
curr_field

,ID,Acres,BSI,county,CropGrp,CropTyp,DataSrc,doy,EVI,ExctAcr,...,Notes,PSRI,RtCrpTy,Shap_Ar,Shp_Lng,system_start_time,TRS,image_year,B8,SF_year
0,88304_WSDA_SF_2015,122,-0.342460,Adams,Cereal Grain,wheat,nass,260.0,1.5,122.096627,...,NaN,-0.074292,"Bean, Dry",494107.5202,2496.698284,1.442603e+12,T15R31E16,2015.0,0.600546,2015
1,88304_WSDA_SF_2015,122,-0.458529,Adams,Cereal Grain,wheat,nass,270.0,1.5,122.096627,...,NaN,-0.081506,"Bean, Dry",494107.5202,2496.698284,1.443467e+12,T15R31E16,2015.0,0.550087,2015


In [15]:
# print ("print(curr_field.shape")
# print(curr_field.shape)
# print ("__________________________________________")
################################################################
no_Outlier_TS = rc.interpolate_outliers_EVI_NDVI(outlier_input = curr_field, given_col = indeks)

In [16]:
no_Outlier_TS

,ID,Acres,BSI,county,CropGrp,CropTyp,DataSrc,doy,EVI,ExctAcr,...,Notes,PSRI,RtCrpTy,Shap_Ar,Shp_Lng,system_start_time,TRS,image_year,B8,SF_year


In [ ]:
output_df[row_pointer: row_pointer + curr_field.shape[0]] = no_Outlier_TS.values
counter += 1
row_pointer += curr_field.shape[0]

In [ ]:
output_df = pd.DataFrame(data = None,
                         index = np.arange(an_EE_TS.shape[0]), 
                         columns = an_EE_TS.columns)

counter = 0
row_pointer = 0

for a_poly in polygon_list:

    print (counter)
    print (a_poly)
    curr_field = an_EE_TS[an_EE_TS['ID']==a_poly].copy()
    ################################################################
    # Sort by DoY (sanitary check)
    curr_field.sort_values(by=['image_year', 'doy'], inplace=True)
    curr_field.reset_index(drop=True, inplace=True)
    
    # print ("print(curr_field.shape")
    # print(curr_field.shape)
    # print ("__________________________________________")
    ################################################################
    no_Outlier_TS = rc.interpolate_outliers_EVI_NDVI(outlier_input = curr_field, given_col = indeks)

    output_df[row_pointer: row_pointer + curr_field.shape[0]] = no_Outlier_TS.values
    counter += 1
    row_pointer += curr_field.shape[0]

In [ ]:
output_df.shape[0]

In [ ]:
len(output_df)

In [ ]:
output_df.head(2)

In [ ]:
given_col = "EVI"

In [ ]:
output_df = output_df[output_df[given_col] < 1.5]

In [ ]:
output_df.shape

In [ ]:
len(output_df)

In [17]:
# Check SF_year type after removing outliers and jumps

In [18]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/test_aeolus_outputs/"
f_name = "Eastern_WA_SF_2016_70cloud_EVI.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

In [21]:
type(a_df.SF_year[0])

numpy.int64

In [28]:
not('SF_year' in a_df.columns)

False